In [1]:
import csv
import pandas as pd
import re

In [2]:
data = pd.read_csv("messages.csv")
data.head(20)

,time,date,message
0,10:14:20,2024-02-22,⚠️ Активність бортів Ту-22м3 східніше в рф. \n...
1,08:28:33,2024-02-22,⚪️ Відбій загрози балістики.
2,08:08:51,2024-02-22,Загроза балістики з Півдня.
3,02:37:25,2024-02-22,"⚡️Брянськ, рф — робота ворожого ППО по БпЛА."
4,22:05:13,2024-02-21,📡 Обстановка станом на 00:00 22.02.24\n\n— Стр...
5,20:39:32,2024-02-21,Дорозвідка по областях.
6,20:19:13,2024-02-21,"Миргород — уважно, рух ударних БпЛА вздовж міста."
7,19:53:24,2024-02-21,3х БпЛА — Полтавська область. Курс Північно-За...
8,19:08:48,2024-02-21,21:08\n\n2х БпЛА — Криворізький район Дніпропе...
9,18:35:25,2024-02-21,Загроза ударних БпЛА зберігається для Дніпропе...


In [3]:
data.shape

(14603, 3)

In [4]:
#remove blank messages
data = data[~data["message"].isna()]
data.shape

(14364, 3)

In [5]:
def clean_text(text):
    #remove emojis and special symbols
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  #emoticons
                               u"\U0001F300-\U0001F5FF"  #symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  #transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  #flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r'[\n\t\r]', ' ', text)
    #remove mention "@war_monitor"
    text = re.sub(r'@war_monitor', '', text)
    return text

In [6]:
data['message'] = data['message'].astype(str).apply(clean_text)
data.head(20)

,time,date,message
0,10:14:20,2024-02-22,Активність бортів Ту-22м3 східніше в рф. Заг...
1,08:28:33,2024-02-22,Відбій загрози балістики.
2,08:08:51,2024-02-22,Загроза балістики з Півдня.
3,02:37:25,2024-02-22,"Брянськ, рф — робота ворожого ППО по БпЛА."
4,22:05:13,2024-02-21,Обстановка станом на 00:00 22.02.24 — Страте...
5,20:39:32,2024-02-21,Дорозвідка по областях.
6,20:19:13,2024-02-21,"Миргород — уважно, рух ударних БпЛА вздовж міста."
7,19:53:24,2024-02-21,3х БпЛА — Полтавська область. Курс Північно-За...
8,19:08:48,2024-02-21,21:08 2х БпЛА — Криворізький район Дніпропетр...
9,18:35:25,2024-02-21,Загроза ударних БпЛА зберігається для Дніпропе...


In [7]:
#aggregate data for same day to one text. drop time column then
data['date'] = pd.to_datetime(data['date'])
data_grouped = data.groupby('date')['message'].apply(lambda x: ' '.join(x)).reset_index()
data_grouped.tail(20)

,date,message
639,2024-02-03,Обстановка станом на 00:00 04.01.24 — Страте...
640,2024-02-04,Загроза балістики з Бєлгородської області рф. ...
641,2024-02-05,Обстановка станом на 00:00 06.02.24 — Страте...
642,2024-02-06,01:50 Шахеди продовжують рухатись у напрямку ...
643,2024-02-07,Миколаїв — перебувайте в укриттях. Ударні БпЛА...
644,2024-02-08,Дорозвідка по БпЛА. 00:27 Група 2: 2х БпЛА. Х...
645,2024-02-09,Останні БпЛА у бік населених пунктів Ізмаїльсь...
646,2024-02-10,01:56 Київ — загроза ударних БпЛА. Вінницька...
647,2024-02-11,01:45 Два ударних БпЛА вздовж кордону Вінниць...
648,2024-02-12,Відбій загрози ударних БпЛА. Дніпро — чисто. ...


In [8]:
#checking 
desired_date = '2024-02-22'  
selected_data = data_grouped[data_grouped['date'] == desired_date]
selected_data['message'].values[0]

' Активність бортів Ту-22м3 східніше в рф.  Загроза застосування ракет Х-22.  Відбій загрози балістики. Загроза балістики з Півдня. Брянськ, рф — робота ворожого ППО по БпЛА.'

In [9]:
desired = '2024-02-21'  
selected = data_grouped[data_grouped['date'] == desired]
selected['message'].values[0]

' Обстановка станом на 00:00 22.02.24  — Стратегічна авіація: Триває передислокація бортів Ту-95 та Ту-22м3 в межах Західного регіону рф;  — БпЛА: Потворний запуск ударних БпЛА не фіксувався;  — Флот: Ракетоносії в портах базування.  #обстановка |  Дорозвідка по областях. Миргород — уважно, рух ударних БпЛА вздовж міста. 3х БпЛА — Полтавська область. Курс Північно-Західний. 21:08  2х БпЛА — Криворізький район Дніпропетровщини, курс Північно-Західний.  4х БпЛА —Криворізький→Кам\'янський район Дніпропетровської області. Курс Північний.  1х БпЛА — Красноградський район Харківщини. Курс на Полтавщину. Загроза ударних БпЛА зберігається для Дніпропетровської області, Синельниківський та Дніпровський райони. Не спостерігається.  Ракетна небезпека триває до відбою. Пуск ракети (невст) у напрямку Одеської області.  Загроза ударних БпЛА з південного напрямку.  Запорізька область.  Відбій по областях.  Відхід МіГ-ів від пускових рубежів.  Зліт МіГ-31К ВПС рф.  МіГ-31К — носій надзвукової ракети Х

In [ ]:
#maybe create additional column with cities/oblast mentioned in text for every day